In [1]:
import torch
import torch.nn as nn
import numpy as np

/home/wsjc/anaconda3/envs/sci/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class DepthwiseSeparableConv2d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, padding, dilation):
        super(DepthwiseSeparableConv2d, self).__init__()
        # 深度可分离卷积由深度卷积和逐点卷积两部分组成，先定义深度卷积层
        self.depthwise = nn.Conv2d(in_channels, in_channels, kernel_size=kernel_size, stride=1, padding=padding,
                                   groups=in_channels, dilation=dilation, bias=False)
        # 然后定义逐点卷积层
        self.pointwise = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, padding=0, groups=1, dilation=dilation, bias=False)
        # 用Xavier初始化权重，提高模型收敛速度
        nn.init.xavier_uniform_(self.depthwise.weight)
        nn.init.xavier_uniform_(self.pointwise.weight)

    def forward(self, x):
        # 先进行深度卷积
        x = self.depthwise(x)
        # 再进行逐点卷积
        x = self.pointwise(x)
        return x

In [3]:
class BasicConv2d(nn.Module):
    def __init__(self, in_planes, out_planes, kernel_size, stride=1, padding=0, dilation=1):
        super(BasicConv2d, self).__init__()
        self.conv = nn.Conv2d(in_planes, out_planes,
                              kernel_size=kernel_size, stride=stride,
                              padding=padding, dilation=dilation, bias=False)
        self.bn = nn.BatchNorm2d(out_planes)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        return x

In [4]:
class Step(nn.Module):
    def __init__(self, in_channel, out_channel):
        super(Step, self).__init__()
        self.relu = nn.ReLU(True)
        self.up_m = nn.Sequential(
            nn.AdaptiveAvgPool2d(in_channel),
            BasicConv2d(in_channel, out_channel, kernel_size=3, dilation=5),
            BasicConv2d(out_channel, out_channel, kernel_size=1),
            nn.Upsample(scale_factor=2, mode='nearest')
        )
        self.down_m_0 = nn.Sequential(
            BasicConv2d(in_channel, out_channel, kernel_size=1),
            BasicConv2d(out_channel, out_channel, kernel_size=3, padding=1, dilation=1)
        )
        self.down_m_1 = nn.Sequential(
            BasicConv2d(out_channel, out_channel, kernel_size=1),
            DepthwiseSeparableConv2d(out_channel, out_channel, kernel_size=3),
            BasicConv2d(out_channel, out_channel, kernel_size=3, padding=1, dilation=3)
        )
        self.down_m_2 = nn.Sequential(
            BasicConv2d(out_channel, out_channel, kernel_size=1),
            DepthwiseSeparableConv2d(out_channel, out_channel, kernel_size=5),
            BasicConv2d(out_channel, out_channel, kernel_size=3, padding=1, dilation=5)
        )
        self.down_m_3 = nn.Sequential(
            BasicConv2d(out_channel, out_channel, kernel_size=1),
            DepthwiseSeparableConv2d(out_channel, out_channel, kernel_size=7),
            BasicConv2d(out_channel, out_channel, kernel_size=3, padding=1, dilation=7)
        )
        self.conv_cat = BasicConv2d(4*out_channel, out_channel, 1, padding=1)

    def forward(self, x):
        up_res = self.up_m(x)
        print(up_res.shape)
        down_res_0 = self.down_m_0(x)
        print(down_res_0.shape)
        down_res_1 = self.down_m_1(down_res_0)
        print(down_res_1.shape)
        down_res_2 = self.down_m_2(down_res_1)
        print(down_res_2.shape)
        down_res_3 = self.down_m_3(down_res_2)
        print(down_res_3.shape)
        x_cat = self.conv_cat(torch.cat((down_res_0, down_res_1, down_res_2, down_res_3), 1))

        output = self.relu(x_cat + up_res)
        return output

In [5]:
net = Step(512, 128)

In [6]:
from torchsummary import summary
summary(net, (512, 256, 256))

torch.Size([2, 128, 256, 256])
torch.Size([2, 128, 256, 256])
torch.Size([2, 128, 256, 256])
torch.Size([2, 128, 256, 256])
torch.Size([2, 128, 256, 256])
Layer (type:depth-idx)                        Output Shape              Param #
├─Sequential: 1-1                             [-1, 128, 256, 256]       --
|    └─AdaptiveAvgPool2d: 2-1                 [-1, 512, 128, 128]       --
|    └─BasicConv2d: 2-2                       [-1, 128, 128, 128]       --
|    |    └─Conv2d: 3-1                       [-1, 128, 128, 128]       65,536
|    |    └─BatchNorm2d: 3-2                  [-1, 128, 128, 128]       256
|    └─Upsample: 2-3                          [-1, 128, 256, 256]       --
├─Sequential: 1-2                             [-1, 128, 256, 256]       --
|    └─BasicConv2d: 2-4                       [-1, 128, 256, 256]       --
|    |    └─Conv2d: 3-3                       [-1, 128, 256, 256]       65,536
|    |    └─BatchNorm2d: 3-4                  [-1, 128, 256, 256]       256
|    

Layer (type:depth-idx)                        Output Shape              Param #
├─Sequential: 1-1                             [-1, 128, 256, 256]       --
|    └─AdaptiveAvgPool2d: 2-1                 [-1, 512, 128, 128]       --
|    └─BasicConv2d: 2-2                       [-1, 128, 128, 128]       --
|    |    └─Conv2d: 3-1                       [-1, 128, 128, 128]       65,536
|    |    └─BatchNorm2d: 3-2                  [-1, 128, 128, 128]       256
|    └─Upsample: 2-3                          [-1, 128, 256, 256]       --
├─Sequential: 1-2                             [-1, 128, 256, 256]       --
|    └─BasicConv2d: 2-4                       [-1, 128, 256, 256]       --
|    |    └─Conv2d: 3-3                       [-1, 128, 256, 256]       65,536
|    |    └─BatchNorm2d: 3-4                  [-1, 128, 256, 256]       256
|    └─BasicConv2d: 2-5                       [-1, 128, 256, 256]       --
|    |    └─Conv2d: 3-5                       [-1, 128, 256, 256]       147,456
|    

In [6]:
# TEM
class TEM(nn.Module):
    def __init__(self, in_channel, out_channel):
        super(TEM, self).__init__()
        self.relu = nn.ReLU(True)
        self.branch0 = nn.Sequential(
            BasicConv2d(in_channel, out_channel, 1),
        )
        self.branch1 = nn.Sequential(
            BasicConv2d(in_channel, out_channel, 1),
            BasicConv2d(out_channel, out_channel, kernel_size=(1, 3), padding=(0, 1)),
            BasicConv2d(out_channel, out_channel, kernel_size=(3, 1), padding=(1, 0)),
            BasicConv2d(out_channel, out_channel, 3, padding=3, dilation=3)
        )
        self.branch2 = nn.Sequential(
            BasicConv2d(in_channel, out_channel, 1),
            BasicConv2d(out_channel, out_channel, kernel_size=(1, 5), padding=(0, 2)),
            BasicConv2d(out_channel, out_channel, kernel_size=(5, 1), padding=(2, 0)),
            BasicConv2d(out_channel, out_channel, 3, padding=5, dilation=5)
        )
        self.branch3 = nn.Sequential(
            BasicConv2d(in_channel, out_channel, 1),
            BasicConv2d(out_channel, out_channel, kernel_size=(1, 7), padding=(0, 3)),
            BasicConv2d(out_channel, out_channel, kernel_size=(7, 1), padding=(3, 0)),
            BasicConv2d(out_channel, out_channel, 3, padding=7, dilation=7)
        )
        self.conv_cat = BasicConv2d(4*out_channel, out_channel, 3, padding=1)
        self.conv_res = BasicConv2d(in_channel, out_channel, 1)

    def forward(self, x):
        x0 = self.branch0(x)
        x1 = self.branch1(x)
        x2 = self.branch2(x)
        x3 = self.branch3(x)
        x_cat = self.conv_cat(torch.cat((x0, x1, x2, x3), 1))

        x = self.relu(x_cat + self.conv_res(x))
        return x

In [7]:
net2 = TEM(512, 128)

In [9]:
from torchsummary import summary
summary(net2, (512, 256, 256))

Layer (type:depth-idx)                   Output Shape              Param #
├─Sequential: 1-1                        [-1, 128, 256, 256]       --
|    └─BasicConv2d: 2-1                  [-1, 128, 256, 256]       --
|    |    └─Conv2d: 3-1                  [-1, 128, 256, 256]       65,536
|    |    └─BatchNorm2d: 3-2             [-1, 128, 256, 256]       256
├─Sequential: 1-2                        [-1, 128, 256, 256]       --
|    └─BasicConv2d: 2-2                  [-1, 128, 256, 256]       --
|    |    └─Conv2d: 3-3                  [-1, 128, 256, 256]       65,536
|    |    └─BatchNorm2d: 3-4             [-1, 128, 256, 256]       256
|    └─BasicConv2d: 2-3                  [-1, 128, 256, 256]       --
|    |    └─Conv2d: 3-5                  [-1, 128, 256, 256]       49,152
|    |    └─BatchNorm2d: 3-6             [-1, 128, 256, 256]       256
|    └─BasicConv2d: 2-4                  [-1, 128, 256, 256]       --
|    |    └─Conv2d: 3-7                  [-1, 128, 256, 256]       49,

Layer (type:depth-idx)                   Output Shape              Param #
├─Sequential: 1-1                        [-1, 128, 256, 256]       --
|    └─BasicConv2d: 2-1                  [-1, 128, 256, 256]       --
|    |    └─Conv2d: 3-1                  [-1, 128, 256, 256]       65,536
|    |    └─BatchNorm2d: 3-2             [-1, 128, 256, 256]       256
├─Sequential: 1-2                        [-1, 128, 256, 256]       --
|    └─BasicConv2d: 2-2                  [-1, 128, 256, 256]       --
|    |    └─Conv2d: 3-3                  [-1, 128, 256, 256]       65,536
|    |    └─BatchNorm2d: 3-4             [-1, 128, 256, 256]       256
|    └─BasicConv2d: 2-3                  [-1, 128, 256, 256]       --
|    |    └─Conv2d: 3-5                  [-1, 128, 256, 256]       49,152
|    |    └─BatchNorm2d: 3-6             [-1, 128, 256, 256]       256
|    └─BasicConv2d: 2-4                  [-1, 128, 256, 256]       --
|    |    └─Conv2d: 3-7                  [-1, 128, 256, 256]       49,

In [4]:
#RFB
class RFB(nn.Module):
    def __init__(self, in_channel, out_channel):
        super(RFB, self).__init__()
        self.relu = nn.ReLU(True)
        self.branch0 = nn.Sequential(
            BasicConv2d(in_channel, out_channel, 1),
            BasicConv2d(out_channel, out_channel, 3, padding=1, dilation=1)
        )
        self.branch1 = nn.Sequential(
            BasicConv2d(in_channel, out_channel, 1),
            BasicConv2d(out_channel, out_channel, kernel_size=(3, 1), padding=(1, 0)),
            BasicConv2d(out_channel, out_channel, 3, padding=3, dilation=3)
        )
        self.branch2 = nn.Sequential(
            BasicConv2d(in_channel, out_channel, 1),
            BasicConv2d(out_channel, out_channel, kernel_size=(1, 3), padding=(0, 1)),
            BasicConv2d(out_channel, out_channel, 3, padding=3, dilation=3)
        )
        self.branch3 = nn.Sequential(
            BasicConv2d(in_channel, out_channel, 1),
            BasicConv2d(out_channel, out_channel, kernel_size=(1, 3), padding=(0, 1)),
            BasicConv2d(out_channel, out_channel, kernel_size=(3, 1), padding=(1, 0)),
            BasicConv2d(out_channel, out_channel, 3, padding=5, dilation=5)
        )
        self.conv_cat = BasicConv2d(4*out_channel, out_channel, 1)
        self.conv_res = BasicConv2d(in_channel, out_channel, 1)

    def forward(self, x):
        x0 = self.branch0(x)
        print(x0.shape)
        x1 = self.branch1(x)
        print(x1.shape)
        x2 = self.branch2(x)
        print(x2.shape)
        x3 = self.branch3(x)
        print(x3.shape)
        x_cat = self.conv_cat(torch.cat((x0, x1, x2, x3), 1))
        print(self.conv_res(x).shape)
        x = self.relu(x_cat + self.conv_res(x))
        return x

In [5]:
net1 = RFB(512, 128)

In [6]:
from torchsummary import summary
summary(net1, (512, 256, 256))

torch.Size([2, 128, 256, 256])
torch.Size([2, 128, 256, 256])
torch.Size([2, 128, 256, 256])
torch.Size([2, 128, 256, 256])
torch.Size([2, 128, 256, 256])
Layer (type:depth-idx)                   Output Shape              Param #
├─Sequential: 1-1                        [-1, 128, 256, 256]       --
|    └─BasicConv2d: 2-1                  [-1, 128, 256, 256]       --
|    |    └─Conv2d: 3-1                  [-1, 128, 256, 256]       65,536
|    |    └─BatchNorm2d: 3-2             [-1, 128, 256, 256]       256
|    └─BasicConv2d: 2-2                  [-1, 128, 256, 256]       --
|    |    └─Conv2d: 3-3                  [-1, 128, 256, 256]       147,456
|    |    └─BatchNorm2d: 3-4             [-1, 128, 256, 256]       256
├─Sequential: 1-2                        [-1, 128, 256, 256]       --
|    └─BasicConv2d: 2-3                  [-1, 128, 256, 256]       --
|    |    └─Conv2d: 3-5                  [-1, 128, 256, 256]       65,536
|    |    └─BatchNorm2d: 3-6             [-1, 128, 256,

Layer (type:depth-idx)                   Output Shape              Param #
├─Sequential: 1-1                        [-1, 128, 256, 256]       --
|    └─BasicConv2d: 2-1                  [-1, 128, 256, 256]       --
|    |    └─Conv2d: 3-1                  [-1, 128, 256, 256]       65,536
|    |    └─BatchNorm2d: 3-2             [-1, 128, 256, 256]       256
|    └─BasicConv2d: 2-2                  [-1, 128, 256, 256]       --
|    |    └─Conv2d: 3-3                  [-1, 128, 256, 256]       147,456
|    |    └─BatchNorm2d: 3-4             [-1, 128, 256, 256]       256
├─Sequential: 1-2                        [-1, 128, 256, 256]       --
|    └─BasicConv2d: 2-3                  [-1, 128, 256, 256]       --
|    |    └─Conv2d: 3-5                  [-1, 128, 256, 256]       65,536
|    |    └─BatchNorm2d: 3-6             [-1, 128, 256, 256]       256
|    └─BasicConv2d: 2-4                  [-1, 128, 256, 256]       --
|    |    └─Conv2d: 3-7                  [-1, 128, 256, 256]       49